<a href="https://colab.research.google.com/github/brian-loos/Intership-Projects/blob/main/Style%20Transfer/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
#!pip install --quiet "pytorch-lightning>=1.3" "torchmetrics>=0.3" "torch>=1.6, <1.9" "torchvision"
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install pytorch-lightning
!pip install pickle5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl
import torchvision
import torchvision.models as visionmodels
import torchvision.transforms as transforms
import torch.utils.data.dataloader as DataLoader
import torch.utils.data.dataset as Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable,grad
from torchvision import datasets
from collections import *
import os
from torchvision.io import read_image
from datetime import datetime
import glob
from zipfile import ZipFile
from PIL import Image
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)
pwd = '/content/'
base_dir = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer'
os.chdir(base_dir) 
import models 
os.chdir(pwd)

#Globals

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

FLAGS = {} 
FLAGS['datadir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats'
#FLAGS['labeldir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/landscapes.csv'
FLAGS['labeldir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats_small.csv'
FLAGS['styledir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings'
FLAGS['styleanno'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/paintings.csv'
FLAGS['batch_size'] = 16
FLAGS['test_batch_size'] = 16
FLAGS['learning_rate'] = .0005
FLAGS['num_epochs'] = 2000
FLAGS['im_channels'] = 3
FLAGS['noise_scale_factor'] = .01
FLAGS['style_weight'] = 2e5
FLAGS['pixel_weight'] = 10. 
FLAGS['content_weight'] = 1.
FLAGS['tv_weight'] = 1. 
FLAGS['im_size'] = 256
FLAGS['val_image_path'] = ['/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats/cat-2083492_1280 - Copy (2).jpg',
                           '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats/3683 - Copy (2).jpg',
                           '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats/cat-read-to-pounce-julie-austin-photography - Copy (2).jpg',
                           '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/cats/cat1.jpg'
                            ]
FLAGS['style_image_path'] = ['/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings/man_with_hat.jpg',
                             '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings/wave.jpg',
                             '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings/circle_geometric.jpg',
                             '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/data/Paintings/strokes.jpg']
FLAGS['val_size_mult'] = 2
FLAGS['output_path'] = 'outputs/Texture_net_BN'
FLAGS['model_path'] = 'models/Texture_net_BN'
FLAGS['output_fname'] = 'test_output_'
FLAGS['model_fname'] = 'landscape_model'
FLAGS['home_dir'] = '/content/gdrive/MyDrive/Colab Notebooks/Style Transfer'

#Shortcuts

In [ ]:
'''
  LITDataModule(train_batch_size = , 
                val_batch_size = ,
                train_im_size = , 
                val_im_size = , 
                data_zip = [], 
                data_path = [], 
                train_anno_path = , 
                test_anno_path = , 
                val_anno_path = 
                )
  LITAdaINDatamodule(train_batch_size = , 
                val_batch_size = ,
                train_im_size = , 
                val_im_size = , 
                data_zip = [], 
                data_path = [], 
                train_anno_path = , 
                test_anno_path = , 
                val_anno_path = 
                )
  LITTextureNet(style_image : torch.Tensor,
                      batch_size : int = 8,
                      normalization_method: str = 'BatchNorm2d', 
                      num_layers : int = 6, 
                      base_feat_maps: int = 8, 
                      im_channels: int = 3,
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1e5, 
                      content_weight = 1.)
  LITFastNST(style_image : torch.Tensor,
                      batch_size : int = 8,
                      normalization_method: str = 'BatchNorm2d',
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1., 
                      content_weight = 1., 
                      pixel_weight = 10., 
                      tv_weight = 1.)
  LITAdaIN(batch_size : int = 8,
                      vgg_net : str = 'vgg19', 
                      optimizer : str = 'Adam', 
                      learning_rate : float = .1, 
                      betas : tuple = (.5,.99),
                      noise_scale_factor = .01,
                      style_weight = 1., 
                      content_weight = 1. )
    
  extract_features(data_dir,
                  output_fname,
                  vgg_net = 
                  im_size =   
                  )
  
'''

"\n  LITDataModule(train_batch_size = , \n                val_batch_size = ,\n                train_im_size = , \n                val_im_size = , \n                data_zip = [], \n                data_path = [], \n                train_anno_path = , \n                test_anno_path = , \n                val_anno_path = \n                )\n  LITAdaINDatamodule(train_batch_size = , \n                val_batch_size = ,\n                train_im_size = , \n                val_im_size = , \n                data_zip = [], \n                data_path = [], \n                train_anno_path = , \n                test_anno_path = , \n                val_anno_path = \n                )\n  LITTextureNet(style_image : torch.Tensor,\n                      batch_size : int = 8,\n                      normalization_method: str = 'BatchNorm2d', \n                      num_layers : int = 6, \n                      base_feat_maps: int = 8, \n                      im_channels: int = 3,\n               

#Test Area

##Unzip Data

In [ ]:
# models.unzip_data(['data/cats','data/monet'],['/content/gdrive/MyDrive/Colab Notebooks/data/cats.zip', '/content/gdrive/MyDrive/Colab Notebooks/data/monet_jpg.zip'])


##Import data module

In [ ]:
'''
  Load Data Module for every other Module: Loads single style and multiple examples to train on 
'''
dm = models.LITDataModule(train_batch_size = FLAGS['batch_size'] , 
                val_batch_size = FLAGS['test_batch_size'],
                train_im_size = FLAGS['im_size'], 
                val_im_size = FLAGS['im_size']*FLAGS['val_size_mult'], 
                data_zip = [], 
                data_path = [FLAGS['datadir'],FLAGS['datadir']], 
                train_anno_path = FLAGS['labeldir'], 
                test_anno_path = FLAGS['labeldir'], 
                val_anno_path = FLAGS['labeldir']
                )

'''
  Load Data Module for the AdaIN method by Huang and Belongie
'''

# dm = models.LITAdaINDataModule(train_batch_size = FLAGS['batch_size'] , 
#                 val_batch_size = FLAGS['test_batch_size'],
#                 train_im_size = FLAGS['im_size'], 
#                 val_im_size = FLAGS['im_size']*FLAGS['val_size_mult'], 
#                 data_zip = [], 
#                 data_path = [FLAGS['datadir'],FLAGS['styledir']], 
#                 train_anno_path = FLAGS['labeldir'],
#                 source_anno_path = FLAGS['styleanno']  
#                 )

'\n  Load Data Module for the AdaIN method by Huang and Belongie\n'

##Define Model

In [ ]:
'''
  need to load in test images and styles
'''
transform = transforms.Compose([ transforms.Resize(FLAGS['im_size']*2), transforms.RandomCrop((FLAGS['im_size'],FLAGS['im_size']))])
val_transform = transforms.Compose([ transforms.Resize(FLAGS['im_size']*FLAGS['val_size_mult']), transforms.CenterCrop((FLAGS['im_size']*FLAGS['val_size_mult'],FLAGS['im_size']*FLAGS['val_size_mult']))])
style_image = torch.Tensor([])
val_image = torch.Tensor([])
for s_im,v_im in zip(FLAGS['style_image_path'], FLAGS['val_image_path']): 
  s_image = transforms.ToTensor()(val_transform(Image.open(s_im).convert('RGB'))).unsqueeze(0)
  v_image = transforms.ToTensor()(val_transform(Image.open(v_im).convert('RGB'))).unsqueeze(0)
  style_image = torch.cat((style_image, s_image), dim = 0) 
  val_image = torch.cat((val_image,v_image), dim = 0) 

'''
  comment next line if using AdaIN, otherwise, keep
'''
style_image = style_image[0].unsqueeze(0) 
val_image = val_image[0].unsqueeze(0) 

'''
  load texture network
'''
# texture_net = models.LITTextureNet(style_image,
#                              val_image,
#                       batch_size = FLAGS['batch_size'],
#                       normalization_method  = 'BatchNorm2d', 
#                       style_weight = 1e9, 
#                       content_weight = 2e3,
#                       learning_rate = .001
#                       )
# texture_net_IN = models.LITTextureNet(style_image,
#                              val_image,
#                       batch_size = FLAGS['batch_size'],
#                       normalization_method  = 'InstanceNorm2d', 
#                       style_weight = 1e9, 
#                       content_weight = 2e3,
#                       learning_rate = .001)
'''
  load johnson et al's model
'''
# johnson_nst = models.LITFastNST(style_image,
#                           val_image,
#                       batch_size = FLAGS['batch_size'],
#                       normalization_method =  'BatchNorm2d',
#                       vgg_net = 'vgg16', 
#                       optimizer  = 'Adam', 
#                       learning_rate  = .001, 
#                       style_weight = 1. , 
#                       content_weight = 1., 
#                       pixel_weight = 10., 
#                       tv_weight = .00001)

'''
  load AdaIN module
'''
AdaIN = models.LITAdaIN(batch_size  = FLAGS['batch_size'],
                      vgg_net  = 'vgg16', 
                      optimizer  = 'Adam', 
                      learning_rate  = 2e-4, 
                      style_weight = 100, 
                      content_weight = 1,
                      test_style = style_image, 
                      test_image = val_image )


##Define Logger and Trainer

In [ ]:
'''
  different tensor board loggers for different kind of modules
'''
#tb_logger_texture_net = pl_loggers.TensorBoardLogger("logs/texture_net")
#tb_logger_nst = pl_loggers.TensorBoardLogger('logs/Fast_NST')
#logger_texture_net_IN = pl_loggers.TensorBoardLogger("logs/texture_net_IN")
ada_logger = pl_loggers.TensorBoardLogger('logs/adain')
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

'''
  Different trainers which load different metrics for different early stopping methods
  and calling different trainers 
'''
# texture_net_trainer = pl.Trainer(gpus = 1, logger = tb_logger_texture_net, log_every_n_steps= 10, stochastic_weight_avg = True, 
#                       callbacks = [EarlyStopping(monitor = 'style_loss',min_delta= .1, patience=25, verbose=True, mode="min", check_finite=True, stopping_threshold=1.)])
# texture_net_trainer_IN = pl.Trainer(gpus = 1, logger = logger_texture_net_IN, log_every_n_steps= 10, stochastic_weight_avg = True, 
#                       callbacks = [EarlyStopping(monitor = 'style_loss',min_delta= .1, patience=25, verbose=True, mode="min", check_finite=True, stopping_threshold=1.)])
# nst_trainer = pl.Trainer(gpus = 1, logger = tb_logger_nst, log_every_n_steps= 10, stochastic_weight_avg = True, 
#                       callbacks = [EarlyStopping(monitor = 'total loss',min_delta= .01, patience=25, verbose=True, mode="min", check_finite=True, stopping_threshold=1.0)])

ada_trainer = pl.Trainer(gpus = 1, logger = ada_logger, log_every_n_steps= 10, stochastic_weight_avg = True, 
                      callbacks = [EarlyStopping(monitor = 'total loss',min_delta= .001, patience=100, verbose=True, mode="min", check_finite=True, stopping_threshold=.1), lr_monitor])


##Run  Model

In [ ]:
'''
  running fit on different trainers to train different models
'''
# texture_net_trainer.fit(texture_net,dm)
# texture_net_trainer_IN.fit(texture_net_IN,dm)
# nst_trainer.fit(johnson_nst,dm) 
# ada_trainer.fit(AdaIN, dm) 

In [ ]:
'''
  Load TensorBoard Logger
'''
%reload_ext tensorboard
%cd /content/
%tensorboard --logdir logs/adain

In [ ]:
'''
  saving different checkpoints for different models
'''
ada_trainer.save_checkpoint('/content/gdrive/MyDrive/Colab Notebooks/Style Transfer/models/LitAdaIn.ckpt')

In [ ]:
'''
  Printing sample output for trained AdaIN module
'''
AdaIN.eval() 
_,_,_,_, samples = AdaIN.texture_net(val_image, style_image)
samples.clip_(1e-8, 1- 1e-8) 
samples = torch.cat((val_image, style_image, samples), dim =0)
grid = torchvision.utils.make_grid(samples, nrow  = 4)
print(grid.shape)
fig = plt.figure(figsize = (10,10), dpi = 200) 
ax = fig.subplots() 
ax.imshow(grid.detach().permute(1,2,0))
plt.axis('off')
plt.show(block = False)